<a href="https://colab.research.google.com/github/scumulder/Machine-Learning-Spring-2019-UCF/blob/master/Homework%202/K_Fold_Validation_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Instructions:**

Reevaluate your best architecture using k-fold validation with k=5, that is, the size of the validation fold is 20%. Does the accuracy/loss obtain by k-fold validation differ from the accuracy/loss obtain by simple hold-out validation.

**Conclusion:**

The accuracy of the Simple Hold-Out model is 77.79% and that of the K-Fold model is 76.97%.
There is not a big difference between the two validation methods, however based on the results, Simple Hold-Out appears to be the better one.

**About the CIFAR-10 Dataset**

The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.

The dataset is divided into **five training batches** and **one test batch**, each with 10000 images, i.e. 10000 x 3072 numpy array. 

Between them, the training batches contain exactly 5000 images from each class. The test batch contains exactly 1000 randomly-selected images from each class.  



**Loading the CIFAR10 data set:**


In [0]:
from keras.datasets import cifar10
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

#X_train = train_images
#Y_train = train_labels

#X_test = test_images
#Y_test = test_labels

Using TensorFlow backend.


170500096/170498071 [==============================] - 8s 0us/step


**Prepare the training and validation image data:**

In [0]:
train_images = train_images.astype('float32')/ 255
test_images = test_images.astype('float32')/ 255

**Prepare the labels:**


In [0]:
from keras.utils.np_utils import to_categorical   

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

print('Training label shape:', train_labels.shape)
print('Test label shape:', test_labels.shape)

Training label shape: (50000, 10)
Test label shape: (10000, 10)


**Convolutional Neural Network Architecture:**

In [0]:
from keras import models
from keras import layers

batch_size = 256
num_classes = 10
epochs = 50

def cnn():

  network = models.Sequential()

  network.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
  network.add(layers.Conv2D(32, (3, 3), activation='relu'))
  network.add(layers.MaxPooling2D(pool_size = (2, 2)))
  network.add(layers.Dropout(0.15))

  network.add(layers.Conv2D(64, (3, 3), activation='relu'))
  network.add(layers.MaxPooling2D(pool_size = (2, 2)))
  network.add(layers.Dropout(0.15))

  network.add(layers.Conv2D(128, (3, 3), activation='relu'))
  network.add(layers.MaxPooling2D(pool_size = (2, 2)))
  network.add(layers.Dropout(0.15))

  network.add(layers.Flatten())


  network.add(layers.Dense(256, activation='relu'))
  network.add(layers.Dropout(0.35))
  network.add(layers.Dense(num_classes, activation='softmax'))
  network.summary()
  
  network.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
  
  return network

In [0]:
#network.fit(train_images, train_labels, batch_size=batch_size, epochs=epochs, verbose=1, validation_split = 0.2)

from keras.wrappers.scikit_learn import KerasClassifier
neural_network = KerasClassifier(build_fn=cnn, 
                                 epochs=epochs, 
                                 batch_size=batch_size, 
                                 verbose=1)

**Train the network:**

In [0]:
from sklearn.model_selection import cross_val_score
from statistics import mean 

scores = cross_val_score(neural_network, train_images, train_labels, cv=5) # cv = number of folds
average = mean(scores)
print('Average accuracy: ', average*100, '%')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_29 (Conv2D)           (None, 30, 30, 32)        896       
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 28, 28, 32)        9248      
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_29 (Dropout)         (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 12, 12, 64)        18496     
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 6, 6, 64)          0         
_________________________________________________________________
dropout_30 (Dropout)         (None, 6, 6, 64)          0         
__________